In [1]:
import os
from dotenv import load_dotenv
import json
from dataclasses import dataclass

from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

from autogen_core import AgentId, MessageContext, RoutedAgent, SingleThreadedAgentRuntime, message_handler

In [2]:
load_dotenv()
# Check if variables are correctly loaded from .env
api_key = os.getenv('AZURE_OPENAI_API_KEY')
if not api_key:
    raise ValueError("AZURE_OPENAI_API_KEY not found in environment variables")

DEPLOYMENT_NAME = os.getenv('DEPLOYMENT_NAME')
if not DEPLOYMENT_NAME:
    raise ValueError("DEPLOYMENT_NAME not found in environment variables")

API_VERSION = os.getenv('API_VERSION')
if not DEPLOYMENT_NAME:
    raise ValueError("API_VERSION not found in environment variables")
    
AZURE_ENDPOINT = os.getenv('AZURE_ENDPOINT')
if not AZURE_ENDPOINT:
    raise ValueError("AZURE_ENDPOINT not found in environment variables")

In [3]:
# Define LLM model
llm = AzureChatOpenAI(
    temperature = 0,
    model_name = "gpt-4o",
    deployment_name = DEPLOYMENT_NAME,  
    api_version = API_VERSION,
    azure_endpoint = AZURE_ENDPOINT
)

### Statefully manage chat history ###
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]
store = {} # This dictionary will retain all chat history
        
# Load company data 
path_json_company = 'data/json_company.json'
with open(path_json_company, 'r', encoding='utf-8') as file:
    json_company = json.load(file)

# Load client data 
path_json_client = 'data/json_client.json'
with open(path_json_client, 'r', encoding='utf-8') as file:
    json_client = json.load(file)

# This class is essential for Autogen, but additional attributes can be added if necessary.
@dataclass
class Message:
    content: str

In [4]:
# Agent to collect information about the company
# The purpose of this agent is to return relevant information in a given format, so it can be used in, for example, a SQL command.
# The user will not see this agent's response, so we do not need to consider the agent's personality.
system_template_company = """ 
You are specialized in internal queries.
You have access to a set of internal information.
When receiving a question from a user, select which of this information is relevant.
Return the names of all relevant keys separated by commas, along with their probabilities separated by ':'. Try to select at least 3 topics.
If no information is relevant, return 'default.'

User question:
{user_question}

Available information:
- company_history: History of your company describing the history of {company_name}.
- location: Location of {company_name}'s branches.
- mission_statement: Mission of the company {company_name}.
- electronics: electronic Products available for sale.
- food: Edible products available for sale.
- promotions: Current ongoing promotions.
"""

# Agent to collect information about the company
class CompanyAgent(RoutedAgent):
    def __init__(self, description, system_template, llm, json_data):
        super().__init__(description) 
        self.system_template = system_template
        self.llm = llm
        self.json_data = json_data

    @message_handler
    async def on_my_message(self, message: Message, ctx: MessageContext) -> Message:
        print("TESTING: Inside CompanyAgent.")

        # Set LLM
        company_name = self.json_data["company_name"]
        system_prompt = ChatPromptTemplate.from_template(self.system_template) 
        chain_company = system_prompt | self.llm
        answer_chain_company = chain_company.invoke(
            {
                'company_name': company_name,
                'user_question': message.content,
            }
        )
        
        print(f"TESTING: answer_chain_company: {answer_chain_company.content}.")
        
        # Get relevant information from json
        relevant_information = ""
        for e in answer_chain_company.content.split(','):
            e_2 = e.split(':')
            if e_2[0].strip() in ['electronics', 'food']:
                relevant_information += "\n"+ self.json_data['available_products'][e_2[0].strip()]
            else:
                value = self.json_data[e_2[0].strip()]
                if value is not None and (len(value) > 0):
                    relevant_information += "\n"+ value

        print(f"TESTING: relevant_information: {relevant_information}.")
        
        print('-----------------------------------------------------------------------')
        print()
        return Message(content = relevant_information)

 

In [5]:
# Agent to collect information about the company
# The purpose of this agent is to return relevant information in a given format, so it can be used in, for example, a SQL command.
# The user will not see this agent's response, so we do not need to consider the agent's personality.
# We can add as many agent as we want. 
system_template_client = """ 
You are specialized in queries.
You have access to a set of information.
When receiving a question from a user, select which of this information is relevant.
Return the names of all relevant keys separated by commas, along with their probabilities separated by ':'. Try to select at least 3 topics.
If no information is relevant, return 'default.'

User question:
{user_question}

Available information:
- address: Address of {client_name}.
- last_purchase_date: Last purchase date of {client_name}.
- last_purchase: Last purchase of {client_name}.
"""

# Agent to collect information about the client
class ClientAgent(RoutedAgent):
    def __init__(self, description, system_template, llm, json_data):
        super().__init__(description)  
        self.system_template = system_template
        self.llm = llm
        self.json_data = json_data

    @message_handler
    async def on_my_message(self, message: Message, ctx: MessageContext) -> Message:

        print("TESTING: Inside ClientAgent.")

        # Set LLM
        client_name = self.json_data["client_name"]
        system_prompt = ChatPromptTemplate.from_template(self.system_template)
        chain_client = system_prompt | self.llm
        answer_chain_client = chain_client.invoke(
            {
                'client_name': client_name,
                'user_question': message.content,
            }
        )
        print(f"TESTING: answer_chain_client: {answer_chain_client.content}.")

        # Get relevant information from json
        relevant_information = ""
        for e in answer_chain_client.content.split(','):
            e_2 = e.split(':')
            value = self.json_data[e_2[0].strip()]
            if value is not None and (len(value) > 0):
                relevant_information += "\n"+ value

        print(f"TESTING: relevant_information: {relevant_information}.")
        print('-----------------------------------------------------------------------')
        print()
        return Message(content=relevant_information)

In [6]:
# This is the agent that will communicate directly with the user.
# Here, we need to consolidate all the information returned by the other agents.
# Additionally, it is crucial to assign a personality to this agent.

system_template_OuterAgent = """ 
You are the helpful virtual assistent VirtAssist working for {company_name}.
You always answer {client_name} questions received using the tones: {company_tone}.
To answer the user question, you take into account company information and the user information.

Company information:
{response_company}

User information:
{response_client}

User question:
{user_question}
"""
# Outer Agent
class OuterAgent(RoutedAgent):
    def __init__(
        self,
        description,
        llm,
        session_id,
        system_template,
        company_agent_id,
        client_agent_id,
        json_company,
        json_client,
    ):
        super().__init__(description)
        self.llm = llm
        self.session_id = session_id
        self.system_template = system_template
        self.company_agent_id = AgentId(company_agent_id, self.id.key)
        self.client_agent_id = AgentId(client_agent_id, self.id.key)
        self.json_company = json_company
        self.json_client = json_client

    @message_handler
    async def on_my_message(self, message: Message, ctx: MessageContext) -> Message:

        print("TESTING: Inside OuterAgent.")

        # Get information from other agents
        response_company = await self.send_message(message, self.company_agent_id)
        response_client = await self.send_message(message, self.client_agent_id)

        # Set LLM 
        company_name = self.json_company["company_name"]
        company_tone = self.json_company.get("tone", "neutral")
        client_name = self.json_client["client_name"]
        # Use this to run chat without history
        # system_prompt = ChatPromptTemplate.from_template(self.system_template)
        # # Create a chain for question rewriting
        # chain_OuterAgent = system_prompt | self.llm
        
        # answer_chain_OuterAgent = chain_OuterAgent.invoke(
        #     {
        #         'company_name': company_name,
        #         'company_tone': company_tone,
        #         'client_name': client_name,
        #         "response_company": response_company,
        #         "response_client": response_client,
        #         'user_question': message.content,
        #     }
        # )

        # Use this to add chat history
        system_prompt = ChatPromptTemplate.from_messages([
            ("system", self.system_template),
            MessagesPlaceholder(variable_name="chat_history"),
            ("human", "{user_question}"),
        ])
        chain_OuterAgent = system_prompt | self.llm
        ### Statefully manage chat history ###
        conversational_chain_OuterAgent = RunnableWithMessageHistory(
            chain_OuterAgent,
            get_session_history,
            input_messages_key="user_question",
            history_messages_key="chat_history",
        )

        answer_chain_OuterAgent = conversational_chain_OuterAgent.invoke(
            {
                # Variables used in prompt
                'company_name': company_name,
                'company_tone': company_tone,
                'client_name': client_name,
                "response_company": response_company,
                "response_client": response_client,
                'user_question': message.content, 
            },
            config={
                "configurable": {"session_id": self.session_id}
            },  # constructs a key "session_id" in `store`.
        ) 

        print(f"TESTING: answer_chain_OuterAgent: {answer_chain_OuterAgent.content}")
        print('-----------------------------------------------------------------------')
        print()
        return Message(content=answer_chain_OuterAgent.content)


In [7]:
# initializes a runtime environment for multiple agents, registers them, and starts the runtime.
runtime = SingleThreadedAgentRuntime()
await CompanyAgent.register(runtime, "company_agent", lambda: CompanyAgent(
    description="CompanyAgent",
    system_template=system_template_company,
    llm=llm,
    json_data=json_company
))
await ClientAgent.register(runtime, "client_agent", lambda: ClientAgent(
    description="ClientAgent",
    system_template=system_template_client,
    llm=llm,
    json_data=json_client
))
await OuterAgent.register(
    runtime,
    "outer_agent",
    lambda: OuterAgent(
        description="OuterAgent",
        llm=llm,
        session_id = 'Teste_01',
        system_template=system_template_OuterAgent,
        company_agent_id="company_agent",
        client_agent_id="client_agent",
        json_company=json_company,
        json_client=json_client,
    ),
)

# The runtime.start() command launches the runtime, enabling the agents to function and interact as intended.
runtime.start()

TESTING: Inside OuterAgent.
TESTING: Inside CompanyAgent.
TESTING: answer_chain_company: electronics:0.3, food:0.3, promotions:0.4.
TESTING: relevant_information: 
Available products - TV: 100; Laptops: 400
Available products - Chocolate: 600, Meet: 200
Promotion of the day: Buy 1 chocolate, receive 3..
-----------------------------------------------------------------------

TESTING: Inside ClientAgent.
TESTING: answer_chain_client: last_purchase_date:0.3, last_purchase:0.5, address:0.2.
TESTING: relevant_information: 
Last purchase date: 30 days ago
Last purchase: 30 chocolates
address: São Paulo.
-----------------------------------------------------------------------

TESTING: answer_chain_OuterAgent: Well, Bob, considering your last purchase was a whopping 30 chocolates and it’s been 30 days since then, I’d say it’s time to restock your sweet stash! Plus, with today’s promotion, you buy 1 chocolate and get 3 more! It’s like a chocolate party waiting to happen. 🎉🍫

But hey, if you’re

In [8]:
# Send and receive a message
# Give an id for the agent that will communicate with the user
outer_agent_id = AgentId("outer_agent", "default")

# Send message to chat
user_message = "What should I buy?"
response = await runtime.send_message(Message(content=user_message), outer_agent_id)

print(f"Final response: {response.content}")

Final response: Well, Bob, considering your last purchase was a whopping 30 chocolates and it’s been 30 days since then, I’d say it’s time to restock your sweet stash! Plus, with today’s promotion, you buy 1 chocolate and get 3 more! It’s like a chocolate party waiting to happen. 🎉🍫

But hey, if you’re feeling tech-savvy, we’ve got TVs and laptops too. Just imagine watching your favorite shows on a new TV while munching on all that chocolate. Sounds like a plan, right? 😄📺🍫


In [9]:
# Test chat history
# Send message to chat
user_message = "What party did you say?"
response = await runtime.send_message(Message(content=user_message), outer_agent_id)
print(response.content)

Oh, Bob, you know the best kind of party is a chocolate party! 🎉🍫 But if you’re looking for something a bit more... lively, how about a tech party? Imagine unboxing the latest gadgets, setting up your new TV, and maybe even throwing in a new sound system for good measure. It’s like a rave, but with less dancing and more unwrapping. 🕺📦

So, what’s it gonna be? Chocolate or tech? Or both? Because why not have the best of both worlds! 😄


In [10]:
# Test chat history
# Send message to chat
user_message = "What did you asked me to imagine?"
response = await runtime.send_message(Message(content=user_message), outer_agent_id)
print(response.content)

Ah, Bob, I asked you to imagine the ultimate combo: a chocolate party and a tech party! 🎉🍫📺 Picture this: you, surrounded by a mountain of chocolates, unwrapping the latest gadgets, and setting up your new TV. It’s like Christmas morning, but with more sugar and fewer carols. 😄

So, are you ready to dive into this sweet and techy dream? Or do you need more convincing? Because I’ve got plenty of imagination to spare! 😉


In [12]:
# stop execution once it becomes idle, meaning when all tasks or agents within the runtime have completed their work and there is nothing left to process.
await runtime.stop_when_idle()